# JSON Mode (Yapılandırılmış Çıktı)

## 🎯 Bu Kodun Amacı
AI'dan her zaman tutarlı JSON formatında yanıt almak ve veri analizini yapılandırmak.

## Ne Yapacağız?
- **response_format** parametresi ile JSON formatını zorunlu kılacağız
- Havayolu müşteri verisini pandas ile okuyacağız
- Veri özeti çıkaracağız ve AI'ya analiz ettireceğiz
- AI'dan **belirli bir JSON şeması** ile yanıt alacağız (genel_durum, kritik_bulgular, öneriler)
- JSON çıktısını `json.loads()` ile parse edip kullanacağız

## Çalıştırma
Kod çalıştırıldığında CSV verisi analiz edilir ve JSON formatında yapılandırılmış bir rapor üretilir.

In [1]:
import os
from dotenv import load_dotenv
from groq import Groq
import pandas as pd
import json

load_dotenv()

True

## 📊 Groq Client ve Veri Özeti Hazırlama

**Ne yapıyoruz?**
- Groq API client'ı oluşturuyoruz
- CSV dosyasını pandas ile okuyoruz
- Veri hakkında özet bilgi hazırlıyoruz (toplam kayıt, kolonlar, istatistikler)

**data_summary:**
- AI'ya göndereceğimiz veri özeti
- `len(df)`: Toplam satır sayısı
- `df.columns.tolist()[:10]`: İlk 10 kolon adı
- `df.head(1).to_dict('records')[0]`: İlk satır örneği
- `df.select_dtypes(include=['number']).mean()`: Numerik kolonların ortalamaları

In [3]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

df = pd.read_csv('airline_passenger_satisfaction.csv')

data_summary = f"""
Toplam Kayıt: {len(df)}
Kolonlar: {', '.join(df.columns.tolist()[:10])}
İlk satır örneği:
{df.head(1).to_dict('records')[0]}

Temel İstatistikler:
- Ortalama değerler: {df.select_dtypes(include=['number']).mean().to_dict()}
"""

## 🔧 JSON Mode ile Yapılandırılmış Yanıt Alma

**Ne yapıyoruz?**
- AI'dan veri analizi istiyoruz ama **sadece JSON formatında** yanıt alıyoruz
- `response_format={"type": "json_object"}`: JSON formatını zorunlu kılar

**System prompt:**
- "Sen bir veri analisti asistanısın. Her zaman JSON formatında cevap verirsin."
- AI'ya rolünü ve çıktı formatını söylüyoruz

**User prompt:**
- Veri özetini gönderiyoruz
- İstediğimiz JSON şemasını belirtiyoruz: `genel_durum`, `toplam_kayit`, `kritik_bulgular`, `oneriler`

**json.loads():**
- AI'dan gelen string JSON'u Python dictionary'e çeviriyoruz
- `ensure_ascii=False`: Türkçe karakterleri düzgün göstermek için

**Neden JSON Mode?**
- Tutarlı çıktı formatı garantisi
- Parse edilebilir veri (programla işlenebilir)
- Veritabanına kaydetmek veya frontend'e göndermek için ideal

In [4]:
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": "Sen bir veri analisti asistanısın. Her zaman JSON formatında cevap verirsin."
        },
        {
            "role": "user",
            "content": f"""Aşağıdaki havayolu müşteri verisini analiz et:

{data_summary}

Şu JSON formatında döndür:
{{
    "genel_durum": "açıklama",
    "toplam_kayit": sayı,
    "kritik_bulgular": ["bulgu1", "bulgu2"],
    "oneriler": ["öneri1", "öneri2"]
}}"""
        }
    ],
    response_format={"type": "json_object"},
    temperature=0.3,
    max_tokens=500
)

result = json.loads(response.choices[0].message.content)

print("ANALIZ SONUCU:")
print(json.dumps(result, indent=2, ensure_ascii=False))

ANALIZ SONUCU:
{
  "genel_durum": "Havayolu müşteri verisi analizinde, müşteri memnuniyetini etkileyen faktörler arasında uçuş mesafesi, kalkış ve varış zamanı konforu, online rezervasyon kolaylığı, check-in hizmeti, uçuş içi hizmetler, bagaj işlemleri gibi unsurlar öne çıkmaktadır.",
  "toplam_kayit": 129880,
  "kritik_bulgular": [
    "Ortalama uçuş mesafesi 1190 km'dir.",
    "Ortalama kalkış gecikmesi 14.71 dakika, ortalama varış gecikmesi 15.09 dakika olarak tespit edilmiştir.",
    "Müşterilerin %100'ü 'Neutral or Dissatisfied' olarak tanımlanmıştır, bu durum müşteri memnuniyetinin düşük olduğunu göstermektedir."
  ],
  "oneriler": [
    "Müşteri memnuniyetini artırmak için uçuş içi hizmetlerin iyileştirilmesi",
    "Kalkış ve varış zamanı konforunun artırılması",
    "Online rezervasyon ve check-in süreçlerinin kolaylaştırılması",
    "Bagaj işlemlerinin hızlandırılması ve güvenirliğinin artırılması"
  ]
}
